In [1]:
import paddle
import paddle_quantum
import numpy
import math
import time
import os
from paddle import matmul
from paddle_quantum.linalg import dagger
from paddle_quantum.ansatz import Circuit
from paddle_quantum.qinfo import pauli_str_to_matrix

# Hardware efficient ansatz
def U_hardware(theta, num_qubits, depth):

    # Initialize quantum circuit
    circuit = Circuit(num_qubits)
    
    for j in range(depth):
        circuit.ry(qubits_idx='full', param=theta[j, :, 0])
        circuit.rz(qubits_idx='full', param=theta[j, :, 1])

        circuit.h(qubits_idx=1)
        circuit.cnot(qubits_idx=[0,1])
        circuit.h(qubits_idx=1)

        circuit.h(qubits_idx=2)
        circuit.cnot(qubits_idx=[1,2])
        circuit.h(qubits_idx=2)

        circuit.h(qubits_idx=3)
        circuit.cnot(qubits_idx=[2,3])
        circuit.h(qubits_idx=3)

        circuit.h(qubits_idx=4)
        circuit.cnot(qubits_idx=[3,4])
        circuit.h(qubits_idx=4)

        circuit.h(qubits_idx=5)
        circuit.cnot(qubits_idx=[4,5])
        circuit.h(qubits_idx=5)

        circuit.h(qubits_idx=6)
        circuit.cnot(qubits_idx=[5,6])
        circuit.h(qubits_idx=6)

        circuit.h(qubits_idx=7)
        circuit.cnot(qubits_idx=[6,7])
        circuit.h(qubits_idx=7)

    return circuit

def symmetry_preserving_circuit(circuit, theta, j):
    circuit.rz(qubits_idx=[1, 3, 5, 7], param=math.pi/2)
    
    circuit.cnot(qubits_idx=[[1, 0], [3, 2], [5, 4], [7, 6]])

    circuit.rz(qubits_idx=[0, 2, 4, 6], param=2 * theta[j, 0:4, 0] - math.pi/2)
    circuit.ry(qubits_idx=[1, 3, 5, 7], param=math.pi/2 - 2 * theta[j, 0:4, 0])

    circuit.cnot(qubits_idx=[[0,1], [2,3], [4,5], [6,7]])

    circuit.ry(qubits_idx=[1, 3, 5, 7], param=2 * theta[j, 0:4, 0] - math.pi/2)

    circuit.cnot(qubits_idx=[[1,0], [3,2], [5,4], [7,6]])

    circuit.rz(qubits_idx=[0, 2, 4, 6], param=-math.pi/2)


    circuit.rz(qubits_idx=[2, 4, 6], param=math.pi/2)
    
    circuit.cnot(qubits_idx=[[2, 1], [4, 3], [6, 5]])

    circuit.rz(qubits_idx=[1, 3, 5], param=2 * theta[j, 0:3, 1] - math.pi/2)
    circuit.ry(qubits_idx=[2, 4, 6], param=math.pi/2 - 2 * theta[j, 0:3, 1])

    circuit.cnot(qubits_idx=[[1,2], [3, 4], [5,6]])

    circuit.ry(qubits_idx=[2, 4, 6], param=2 * theta[j, 0:3, 1] - math.pi/2)

    circuit.cnot(qubits_idx=[[2, 1], [4, 3], [6, 5]])

    circuit.rz(qubits_idx=[1, 3, 5], param=-math.pi/2)

    return circuit

# Symmetry ansatz quantum circuit
def U_NumOp(theta, num_qubits, depth):

    # Initialize quantum circuit
    circuit = Circuit(num_qubits)
    
    for j in range(depth):
        symmetry_preserving_circuit(circuit, theta, j)

    return circuit

class VQE_symmetry_class(paddle.nn.Layer):
    def __init__(self, depth, width, length_layer):
        super(VQE_symmetry_class, self).__init__()

        # Random parameters
        Uni_ini = paddle.nn.initializer.Uniform(low=0.0, 
                                                high=2 * math.pi)
        
        self.theta = self.create_parameter(shape=[depth, width, length_layer],
                                           default_initializer=Uni_ini,
                                           dtype='float64')
        
    def forward(self, process_selection, H, NumOp, Sz):
        if process_selection in [Hea_g, Hea_ae, Hea_ae_Sz]:
            cir = U_hardware(self.theta, N, D)
        elif process_selection in [Spa_g, Spa_en3, Spa_en4]:
            cir = U_NumOp(self.theta, N, D)
        
        # Matrix transformation
        U = cir.unitary_matrix()
        H_U = matmul(H, U)
        NumOp_U = matmul(NumOp, U)
        Sz_U = matmul(Sz, U)
        
        Udagger_H_U = matmul(dagger(U), H_U)
        Udagger_NumOp_U = matmul(dagger(U), NumOp_U)
        Udagger_Sz_U = matmul(dagger(U), Sz_U)
        
        H_trans = paddle.real(Udagger_H_U)
        NumOp_trans = paddle.real(Udagger_NumOp_U)
        Sz_trans = paddle.real(Udagger_Sz_U)

        # Define coefficient sign
        def pm(x):
            if x in [0, 2, 4, 6, 8, 10, 13, 15, 17, 19, 20, 22]:
                return 1
            elif x in [1, 3, 5, 7, 9, 11, 12, 14, 16, 18, 21, 23]:
                return -1
        
        # Compute the expectation value of the operator after the initial state undergoes quantum gate operations
        def compute_expectation(basis_states, trans_matrix, coeff):
            exp_psi = 0
            for i in range(len(basis_states)):
                for j in range(len(basis_states)):
                    exp_psi += coeff(i, j) * trans_matrix[basis_states[i]][basis_states[j]]
            return exp_psi

        # The corresponding coefficients of different initial states during the compute process
        def coeff0(i, j): return 1/16
        def coeff1(i, j): return 1/24
        def coeff2(i, j): return 1/24
        def coeff3(i, j): return 1/24 * pm(i) * pm(j)
        def coeff4(i, j): return 1/24 * pm(i) * pm(j)
        def coeff5(i, j): return 1/36

        # The expectation value of the Hamiltonian under different initial states
        H0_psi = compute_expectation(g_values, H_trans, coeff0)
        H1_psi = compute_expectation(e1_values, H_trans, coeff1)
        H2_psi = compute_expectation(e2_values, H_trans, coeff2)
        H3_psi = compute_expectation(e3_values, H_trans, coeff3)
        H4_psi = compute_expectation(e4_values, H_trans, coeff4)
        H5_psi = compute_expectation(e5_values, H_trans, coeff5)

        # The expectation value of the particle number operator under different initial states
        NumOp0_psi = compute_expectation(g_values, NumOp_trans, coeff0)
        NumOp1_psi = compute_expectation(e1_values, NumOp_trans, coeff1)
        NumOp2_psi = compute_expectation(e2_values, NumOp_trans, coeff2)
        NumOp3_psi = compute_expectation(e3_values, NumOp_trans, coeff3)
        NumOp4_psi = compute_expectation(e4_values, NumOp_trans, coeff4)
        NumOp5_psi = compute_expectation(e5_values, NumOp_trans, coeff5)

        # The expectation value of the total spin z component operator under different initial states
        Sz0_psi = compute_expectation(g_values, Sz_trans, coeff0)
        Sz1_psi = compute_expectation(e1_values, Sz_trans, coeff1)
        Sz2_psi = compute_expectation(e2_values, Sz_trans, coeff2)
        Sz3_psi = compute_expectation(e3_values, Sz_trans, coeff3)
        Sz4_psi = compute_expectation(e4_values, Sz_trans, coeff4)
        Sz5_psi = compute_expectation(e5_values, Sz_trans, coeff5)

        H_psi = [H0_psi, H1_psi, H2_psi, H3_psi, 
                 H4_psi, H5_psi]
        
        NumOp_psi = [NumOp0_psi, NumOp1_psi, NumOp2_psi, NumOp3_psi, 
                     NumOp4_psi, NumOp5_psi]
        
        Sz_psi = [Sz0_psi, Sz1_psi, Sz2_psi, Sz3_psi, 
                  Sz4_psi, Sz5_psi]
        
        # Loss function
        loss_g = H_psi[0]
        
        loss_hard_state = 0
        for i in range(len(H_psi)):
            omega = len(H_psi) - i
            loss_hard_state += omega * H_psi[i]

        loss_hard_state_Sz = (6 * H_psi[0] + 5 * (H_psi[1] + beta * (Sz_psi[1] - 0.5)**2) 
                              + 4 * (H_psi[2] + beta * (Sz_psi[2] + 0.5)**2) 
                              + 3 * (H_psi[3] + beta * (Sz_psi[3] + 0.5)**2) 
                              + 2 * (H_psi[4] + beta * (Sz_psi[4] - 0.5)**2) + 1 * H_psi[5])

        loss_n3 = (4 * (H_psi[1] + beta * (Sz_psi[1] - 0.5)**2) 
                   + 3 * (H_psi[2] + beta * (Sz_psi[2] + 0.5)**2) 
                   + 2 * (H_psi[3] + beta * (Sz_psi[3] + 0.5)**2) 
                   + 1 * (H_psi[4] + beta * (Sz_psi[4] - 0.5)**2))

        loss_n4 = H_psi[5]

        if process_selection in [Hea_g, Spa_g]:
            loss = loss_g
        else:
            loss_dict = {
                Hea_ae: loss_hard_state, 
                Hea_ae_Sz: loss_hard_state_Sz, 
                Spa_en3: loss_n3, 
                Spa_en4: loss_n4
            }
            loss = loss_dict[process_selection]
        
        return loss, H_psi, NumOp_psi, Sz_psi, cir

Hea_g = 'Hardware efficient ansatz, ground state'
Hea_ae = 'Hardware efficient ansatz, all eigenstates'
Hea_ae_Sz = 'Hardware efficient ansatz, all eigenstates, Sz penalty term'
Spa_g = 'Symmetry preserving ansatz, ground state'
Spa_en3 = 'Symmetry preserving ansatz, Eigenstate, n=3'
Spa_en4 = 'Symmetry preserving ansatz, Eigenstate, n=4'

states_corresponding = {
    Hea_g: [0], 
    Hea_ae: [i for i in range(6)], 
    Hea_ae_Sz: [i for i in range(6)], 
    Spa_g: [0], 
    Spa_en3: [1, 2, 3, 4], 
    Spa_en4: [5], 
}

file_corresponding = {
    'Hardware efficient ansatz, ground state': 'Hea_g', 
    'Hardware efficient ansatz, all eigenstates': 'Hea_ae', 
    'Hardware efficient ansatz, all eigenstates, Sz penalty term': 'Hea_ae_Sz', 
    'Symmetry preserving ansatz, ground state': 'Spa_g', 
    'Symmetry preserving ansatz, Eigenstate, n=3': 'Spa_en3', 
    'Symmetry preserving ansatz, Eigenstate, n=4': 'Spa_en4'
}

# Select the process to compute
proc_sel = input('Select the process you want to compute. You can choose from (copy one of the lines):\n'
                 '\n'
                 f"{Hea_g}\n"
                 f"{Hea_ae}\n"
                 f"{Hea_ae_Sz}\n"
                 f"{Spa_g}\n"
                 f"{Spa_en3}\n"
                 f"{Spa_en4}\n"
                 '\n'
                 'Note: If any other parameters need to be modified, please adjust them in the code.')

if proc_sel not in states_corresponding:
    print('\n\nInvalid selection. Please choose a valid process.\n\n')
else:
    print(f"You selected: {proc_sel}\n")
    print(f"Program is running... \
    The generated data is saved in the Data_2D_{file_corresponding[proc_sel]} folder on the desktop.\n")

# parameters
t = 1
U = 2
N = 8               # number of qubits
iterations = 500         # number of iterations
Learning_Rate = 0.1      # learning rate
n_file = 5             # number of repeated processes
iter_step_size = 1       # interval of iteration value output
prec = 4            # precision of the observables output
prec_fid = 4        # precision of the fidelity output
beta = 1

if proc_sel in [Hea_g, Hea_ae, Hea_ae_Sz]:
    if proc_sel == Hea_g:
        D = 15      # number of layers in the quantum circuit
    elif proc_sel in [Hea_ae, Hea_ae_Sz]:
        D = 21
        beta = 0.5       # loss function penalty factor
    W = N      # the number of parameters per column in each layer of the quantum circuit
    L = 2      # the number of parameters per row in each layer of the quantum circuit
elif proc_sel in [Spa_g, Spa_en3, Spa_en4]:
    if proc_sel == Spa_g:
        D = 5
    elif proc_sel in [Spa_en3, Spa_en4]:
        D = 7
        beta = 1
    W = int(N/2)
    L = 2

# Initial states
g_1 = '00010001'
g_2 = '00010010'
g_3 = '00010100'
g_4 = '00011000'
g_5 = '00100001'
g_6 = '00100010'
g_7 = '00100100'
g_8 = '00101000'
g_9 = '01000001'
g_10 = '01000010'
g_11 = '01000100'
g_12 = '01001000'
g_13 = '10000001'
g_14 = '10000010'
g_15 = '10000100'
g_16 = '10001000'
g_values = [int(g_1, 2), int(g_2, 2), int(g_3, 2), int(g_4, 2), 
            int(g_5, 2), int(g_6, 2), int(g_7, 2), int(g_8, 2), 
            int(g_9, 2), int(g_10, 2), int(g_11, 2), int(g_12, 2), 
            int(g_13, 2), int(g_14, 2), int(g_15, 2), int(g_16, 2)]
        
e1_1 = '00110001'
e1_2 = '00110010'
e1_3 = '00110100'
e1_4 = '00111000'
e1_5 = '11000001'
e1_6 = '11000010'
e1_7 = '11000100'
e1_8 = '11001000'
e1_9 = '01010001'
e1_10 = '01010010'
e1_11 = '01010100'
e1_12 = '01011000'
e1_13 = '01100001'
e1_14 = '01100010'
e1_15 = '01100100'
e1_16 = '01101000'
e1_17 = '10010001'
e1_18 = '10010010'
e1_19 = '10010100'
e1_20 = '10011000'
e1_21 = '10100001'
e1_22 = '10100010'
e1_23 = '10100100'
e1_24 = '10101000'
e1_values = [int(e1_1, 2), int(e1_2, 2), int(e1_3, 2), int(e1_4, 2), 
             int(e1_5, 2), int(e1_6, 2), int(e1_7, 2), int(e1_8, 2), 
             int(e1_9, 2), int(e1_10, 2), int(e1_11, 2), int(e1_12, 2), 
             int(e1_13, 2), int(e1_14, 2), int(e1_15, 2), int(e1_16, 2), 
             int(e1_17, 2), int(e1_18, 2), int(e1_19, 2), int(e1_20, 2), 
             int(e1_21, 2), int(e1_22, 2), int(e1_23, 2), int(e1_24, 2)]

e2_1 = '00010011'
e2_2 = '00100011'
e2_3 = '01000011'
e2_4 = '10000011'
e2_5 = '00011100'
e2_6 = '00101100'
e2_7 = '01001100'
e2_8 = '10001100'
e2_9 = '00010101'
e2_10 = '00100101'
e2_11 = '01000101'
e2_12 = '10000101'
e2_13 = '00010110'
e2_14 = '00100110'
e2_15 = '01000110'
e2_16 = '10000110'
e2_17 = '00011001'
e2_18 = '00101001'
e2_19 = '01001001'
e2_20 = '10001001'
e2_21 = '00011010'
e2_22 = '00101010'
e2_23 = '01001010'
e2_24 = '10001010'
e2_values = [int(e2_1, 2), int(e2_2, 2), int(e2_3, 2), int(e2_4, 2), 
             int(e2_5, 2), int(e2_6, 2), int(e2_7, 2), int(e2_8, 2), 
             int(e2_9, 2), int(e2_10, 2), int(e2_11, 2), int(e2_12, 2), 
             int(e2_13, 2), int(e2_14, 2), int(e2_15, 2), int(e2_16, 2), 
             int(e2_17, 2), int(e2_18, 2), int(e2_19, 2), int(e2_20, 2), 
             int(e2_21, 2), int(e2_22, 2), int(e2_23, 2), int(e2_24, 2)]

e3_1 = '00010011'
e3_2 = '00100011'
e3_3 = '01000011'
e3_4 = '10000011'
e3_5 = '00011100'
e3_6 = '00101100'
e3_7 = '01001100'
e3_8 = '10001100'
e3_9 = '00010101'
e3_10 = '00100101'
e3_11 = '01000101'
e3_12 = '10000101'
e3_13 = '00010110'
e3_14 = '00100110'
e3_15 = '01000110'
e3_16 = '10000110'
e3_17 = '00011001'
e3_18 = '00101001'
e3_19 = '01001001'
e3_20 = '10001001'
e3_21 = '00011010'
e3_22 = '00101010'
e3_23 = '01001010'
e3_24 = '10001010'
e3_values = [int(e3_1, 2), int(e3_2, 2), int(e3_3, 2), int(e3_4, 2), 
             int(e3_5, 2), int(e3_6, 2), int(e3_7, 2), int(e3_8, 2), 
             int(e3_9, 2), int(e3_10, 2), int(e3_11, 2), int(e3_12, 2), 
             int(e3_13, 2), int(e3_14, 2), int(e3_15, 2), int(e3_16, 2), 
             int(e3_17, 2), int(e3_18, 2), int(e3_19, 2), int(e3_20, 2), 
             int(e3_21, 2), int(e3_22, 2), int(e3_23, 2), int(e3_24, 2)]

e4_1 = '00110001'
e4_2 = '00110010'
e4_3 = '00110100'
e4_4 = '00111000'
e4_5 = '11000001'
e4_6 = '11000010'
e4_7 = '11000100'
e4_8 = '11001000'
e4_9 = '01010001'
e4_10 = '01010010'
e4_11 = '01010100'
e4_12 = '01011000'
e4_13 = '01100001'
e4_14 = '01100010'
e4_15 = '01100100'
e4_16 = '01101000'
e4_17 = '10010001'
e4_18 = '10010010'
e4_19 = '10010100'
e4_20 = '10011000'
e4_21 = '10100001'
e4_22 = '10100010'
e4_23 = '10100100'
e4_24 = '10101000'
e4_values = [int(e4_1, 2), int(e4_2, 2), int(e4_3, 2), int(e4_4, 2), 
             int(e4_5, 2), int(e4_6, 2), int(e4_7, 2), int(e4_8, 2), 
             int(e4_9, 2), int(e4_10, 2), int(e4_11, 2), int(e4_12, 2), 
             int(e4_13, 2), int(e4_14, 2), int(e4_15, 2), int(e4_16, 2), 
             int(e4_17, 2), int(e4_18, 2), int(e4_19, 2), int(e4_20, 2), 
             int(e4_21, 2), int(e4_22, 2), int(e4_23, 2), int(e4_24, 2)]

e5_1 = '00110011'
e5_2 = '00111100'
e5_3 = '00110101'
e5_4 = '00110110'
e5_5 = '00111001'
e5_6 = '00111010'
e5_7 = '11000011'
e5_8 = '11001100'
e5_9 = '11000101'
e5_10 = '11000110'
e5_11 = '11001001'
e5_12 = '11001010'
e5_13 = '01010011'
e5_14 = '01011100'
e5_15 = '01010101'
e5_16 = '01010110'
e5_17 = '01011001'
e5_18 = '01011010'
e5_19 = '01100011'
e5_20 = '01101100'
e5_21 = '01100101'
e5_22 = '01100110'
e5_23 = '01101001'
e5_24 = '01101010'
e5_25 = '10010011'
e5_26 = '10011100'
e5_27 = '10010101'
e5_28 = '10010110'
e5_29 = '10011001'
e5_30 = '10011010'
e5_31 = '10100011'
e5_32 = '10101100'
e5_33 = '10100101'
e5_34 = '10100110'
e5_35 = '10101001'
e5_36 = '10101010'
e5_values = [int(e5_1, 2), int(e5_2, 2), int(e5_3, 2), int(e5_4, 2), 
             int(e5_5, 2), int(e5_6, 2), int(e5_7, 2), int(e5_8, 2), 
             int(e5_9, 2), int(e5_10, 2), int(e5_11, 2), int(e5_12, 2), 
             int(e5_13, 2), int(e5_14, 2), int(e5_15, 2), int(e5_16, 2), 
             int(e5_17, 2), int(e5_18, 2), int(e5_19, 2), int(e5_20, 2), 
             int(e5_21, 2), int(e5_22, 2), int(e5_23, 2), int(e5_24, 2), 
             int(e5_25, 2), int(e5_26, 2), int(e5_27, 2), int(e5_28, 2), 
             int(e5_29, 2), int(e5_30, 2), int(e5_31, 2), int(e5_32, 2), 
             int(e5_33, 2), int(e5_34, 2), int(e5_35, 2), int(e5_36, 2)]

# Define the computational basis
basis = numpy.eye(2**N)

# Used when computing fidelity
initial_state_g = 1/4 * (basis[int(g_1, 2)] + basis[int(g_2, 2)] 
                         + basis[int(g_3, 2)] + basis[int(g_4, 2)] 
                         + basis[int(g_5, 2)] + basis[int(g_6, 2)] 
                         + basis[int(g_7, 2)] + basis[int(g_8, 2)] 
                         + basis[int(g_9, 2)] + basis[int(g_10, 2)] 
                         + basis[int(g_11, 2)] + basis[int(g_12, 2)] 
                         + basis[int(g_13, 2)] + basis[int(g_14, 2)] 
                         + basis[int(g_15, 2)] + basis[int(g_16, 2)])

initial_state_e1 = 1/math.sqrt(24) * (basis[int(e1_1, 2)] + basis[int(e1_2, 2)]
                             + basis[int(e1_3, 2)] + basis[int(e1_4, 2)]
                             + basis[int(e1_5, 2)] + basis[int(e1_6, 2)]
                             + basis[int(e1_7, 2)] + basis[int(e1_8, 2)]
                             + basis[int(e1_9, 2)] + basis[int(e1_10, 2)]
                             + basis[int(e1_11, 2)] + basis[int(e1_12, 2)]
                             + basis[int(e1_13, 2)] + basis[int(e1_14, 2)]
                             + basis[int(e1_15, 2)] + basis[int(e1_16, 2)]
                             + basis[int(e1_17, 2)] + basis[int(e1_18, 2)]
                             + basis[int(e1_19, 2)] + basis[int(e1_20, 2)]
                             + basis[int(e1_21, 2)] + basis[int(e1_22, 2)]
                             + basis[int(e1_23, 2)] + basis[int(e1_24, 2)])

initial_state_e2 = 1/math.sqrt(24) * (basis[int(e2_1, 2)] + basis[int(e2_2, 2)]
                             + basis[int(e2_3, 2)] + basis[int(e2_4, 2)]
                             + basis[int(e2_5, 2)] + basis[int(e2_6, 2)]
                             + basis[int(e2_7, 2)] + basis[int(e2_8, 2)]
                             + basis[int(e2_9, 2)] + basis[int(e2_10, 2)]
                             + basis[int(e2_11, 2)] + basis[int(e2_12, 2)]
                             + basis[int(e2_13, 2)] + basis[int(e2_14, 2)]
                             + basis[int(e2_15, 2)] + basis[int(e2_16, 2)]
                             + basis[int(e2_17, 2)] + basis[int(e2_18, 2)]
                             + basis[int(e2_19, 2)] + basis[int(e2_20, 2)]
                             + basis[int(e2_21, 2)] + basis[int(e2_22, 2)]
                             + basis[int(e2_23, 2)] + basis[int(e2_24, 2)])

initial_state_e3 = 1/math.sqrt(24) * (basis[int(e3_1, 2)] - basis[int(e3_2, 2)]
                             + basis[int(e3_3, 2)] - basis[int(e3_4, 2)]
                             + basis[int(e3_5, 2)] - basis[int(e3_6, 2)]
                             + basis[int(e3_7, 2)] - basis[int(e3_8, 2)]
                             + basis[int(e3_9, 2)] - basis[int(e3_10, 2)]
                             + basis[int(e3_11, 2)] - basis[int(e3_12, 2)]
                             - basis[int(e3_13, 2)] + basis[int(e3_14, 2)]
                             - basis[int(e3_15, 2)] + basis[int(e3_16, 2)]
                             - basis[int(e3_17, 2)] + basis[int(e3_18, 2)]
                             - basis[int(e3_19, 2)] + basis[int(e3_20, 2)]
                             + basis[int(e3_21, 2)] - basis[int(e3_22, 2)]
                             + basis[int(e3_23, 2)] - basis[int(e3_24, 2)])

initial_state_e4 = 1/math.sqrt(24) * (basis[int(e4_1, 2)] - basis[int(e4_2, 2)]
                             + basis[int(e4_3, 2)] - basis[int(e4_4, 2)]
                             + basis[int(e4_5, 2)] - basis[int(e4_6, 2)]
                             + basis[int(e4_7, 2)] - basis[int(e4_8, 2)]
                             + basis[int(e4_9, 2)] - basis[int(e4_10, 2)]
                             + basis[int(e4_11, 2)] - basis[int(e4_12, 2)]
                             - basis[int(e4_13, 2)] + basis[int(e4_14, 2)]
                             - basis[int(e4_15, 2)] + basis[int(e4_16, 2)]
                             - basis[int(e4_17, 2)] + basis[int(e4_18, 2)]
                             - basis[int(e4_19, 2)] + basis[int(e4_20, 2)]
                             + basis[int(e4_21, 2)] - basis[int(e4_22, 2)]
                             + basis[int(e4_23, 2)] - basis[int(e4_24, 2)])

initial_state_e5 = 1/6 * (basis[int(e5_1, 2)] + basis[int(e5_2, 2)]
                             + basis[int(e5_3, 2)] + basis[int(e5_4, 2)]
                             + basis[int(e5_5, 2)] + basis[int(e5_6, 2)]
                             + basis[int(e5_7, 2)] + basis[int(e5_8, 2)]
                             + basis[int(e5_9, 2)] + basis[int(e5_10, 2)]
                             + basis[int(e5_11, 2)] + basis[int(e5_12, 2)]
                             + basis[int(e5_13, 2)] + basis[int(e5_14, 2)]
                             + basis[int(e5_15, 2)] + basis[int(e5_16, 2)]
                             + basis[int(e5_17, 2)] + basis[int(e5_18, 2)]
                             + basis[int(e5_19, 2)] + basis[int(e5_20, 2)]
                             + basis[int(e5_21, 2)] + basis[int(e5_22, 2)]
                             + basis[int(e5_23, 2)] + basis[int(e5_24, 2)]
                             + basis[int(e5_25, 2)] + basis[int(e5_26, 2)]
                             + basis[int(e5_27, 2)] + basis[int(e5_28, 2)]
                             + basis[int(e5_29, 2)] + basis[int(e5_30, 2)]
                             + basis[int(e5_31, 2)] + basis[int(e5_32, 2)]
                             + basis[int(e5_33, 2)] + basis[int(e5_34, 2)]
                             + basis[int(e5_35, 2)] + basis[int(e5_36, 2)])

# The Pauli string representation of the Hamiltonian, particle number operator, and total spin z component operator
H_Pauli = [[-t * 0.5, 'X0,X1,I2,I3,I4,I5,I6,I7'], [-t * 0.5, 'Y0,Y1,I2,I3,I4,I5,I6,I7'], 
           [-t * 0.5, 'X0,Z1,Z2,X3,I4,I5,I6,I7'], [-t * 0.5, 'Y0,Z1,Z2,Y3,I4,I5,I6,I7'], 
           [-t * 0.5, 'I0,X1,X2,I3,I4,I5,I6,I7'], [-t * 0.5, 'I0,Y1,Y2,I3,I4,I5,I6,I7'], 
           [-t * 0.5, 'I0,I1,X2,X3,I4,I5,I6,I7'], [-t * 0.5, 'I0,I1,Y2,Y3,I4,I5,I6,I7'], 
           [-t * 0.5, 'I0,I1,I2,I3,X4,X5,I6,I7'], [-t * 0.5, 'I0,I1,I2,I3,Y4,Y5,I6,I7'], 
           [-t * 0.5, 'I0,I1,I2,I3,X4,Z5,Z6,X7'], [-t * 0.5, 'I0,I1,I2,I3,Y4,Z5,Z6,Y7'], 
           [-t * 0.5, 'I0,I1,I2,I3,I4,X5,X6,I7'], [-t * 0.5, 'I0,I1,I2,I3,I4,Y5,Y6,I7'], 
           [-t * 0.5, 'I0,I1,I2,I3,I4,I5,X6,X7'], [-t * 0.5, 'I0,I1,I2,I3,I4,I5,Y6,Y7'], 
           [U * 0.25, 'I0,I1,I2,I3,I4,I5,I6,I7'], [-U * 0.25, 'I0,I1,I2,I3,Z4,I5,I6,I7'], 
           [-U * 0.25, 'Z0,I1,I2,I3,I4,I5,I6,I7'], [U * 0.25, 'Z0,I1,I2,I3,Z4,I5,I6,I7'], 
           [U * 0.25, 'I0,I1,I2,I3,I4,I5,I6,I7'], [-U * 0.25, 'I0,I1,I2,I3,I4,Z5,I6,I7'], 
           [-U * 0.25, 'I0,Z1,I2,I3,I4,I5,I6,I7'], [U * 0.25, 'I0,Z1,I2,I3,I4,Z5,I6,I7'], 
           [U * 0.25, 'I0,I1,I2,I3,I4,I5,I6,I7'], [-U * 0.25, 'I0,I1,I2,I3,I4,I5,Z6,I7'], 
           [-U * 0.25, 'I0,I1,Z2,I3,I4,I5,I6,I7'], [U * 0.25, 'I0,I1,Z2,I3,I4,I5,Z6,I7'], 
           [U * 0.25, 'I0,I1,I2,I3,I4,I5,I6,I7'], [-U * 0.25, 'I0,I1,I2,I3,I4,I5,I6,Z7'], 
           [-U * 0.25, 'I0,I1,I2,Z3,I4,I5,I6,I7'], [U * 0.25, 'I0,I1,I2,Z3,I4,I5,I6,Z7']]

NumOp_Pauli = [[0.5, 'I0,I1,I2,I3,I4,I5,I6,I7'], [-0.5, 'Z0,I1,I2,I3,I4,I5,I6,I7'], 
            [0.5, 'I0,I1,I2,I3,I4,I5,I6,I7'], [-0.5, 'I0,I1,I2,I3,Z4,I5,I6,I7'], 
            [0.5, 'I0,I1,I2,I3,I4,I5,I6,I7'], [-0.5, 'I0,Z1,I2,I3,I4,I5,I6,I7'], 
            [0.5, 'I0,I1,I2,I3,I4,I5,I6,I7'], [-0.5, 'I0,I1,I2,I3,I4,Z5,I6,I7'], 
            [0.5, 'I0,I1,I2,I3,I4,I5,I6,I7'], [-0.5, 'I0,I1,Z2,I3,I4,I5,I6,I7'], 
            [0.5, 'I0,I1,I2,I3,I4,I5,I6,I7'], [-0.5, 'I0,I1,I2,I3,I4,I5,Z6,I7'], 
            [0.5, 'I0,I1,I2,I3,I4,I5,I6,I7'], [-0.5, 'I0,I1,I2,Z3,I4,I5,I6,I7'], 
            [0.5, 'I0,I1,I2,I3,I4,I5,I6,I7'], [-0.5, 'I0,I1,I2,I3,I4,I5,I6,Z7']]

Sz_Pauli = [[0.25, 'I0,I1,I2,I3,I4,I5,I6,I7'], [-0.25, 'Z0,I1,I2,I3,I4,I5,I6,I7'], 
            [-0.25, 'I0,I1,I2,I3,I4,I5,I6,I7'], [0.25, 'I0,I1,I2,I3,Z4,I5,I6,I7'], 
            [0.25, 'I0,I1,I2,I3,I4,I5,I6,I7'], [-0.25, 'I0,Z1,I2,I3,I4,I5,I6,I7'], 
            [-0.25, 'I0,I1,I2,I3,I4,I5,I6,I7'], [0.25, 'I0,I1,I2,I3,I4,Z5,I6,I7'], 
            [0.25, 'I0,I1,I2,I3,I4,I5,I6,I7'], [-0.25, 'I0,I1,Z2,I3,I4,I5,I6,I7'], 
            [-0.25, 'I0,I1,I2,I3,I4,I5,I6,I7'], [0.25, 'I0,I1,I2,I3,I4,I5,Z6,I7'], 
            [0.25, 'I0,I1,I2,I3,I4,I5,I6,I7'], [-0.25, 'I0,I1,I2,Z3,I4,I5,I6,I7'], 
            [-0.25, 'I0,I1,I2,I3,I4,I5,I6,I7'], [0.25, 'I0,I1,I2,I3,I4,I5,I6,Z7']]

# Matrix representation
dtype = paddle_quantum.get_dtype()
H_matrix = pauli_str_to_matrix(H_Pauli, N).astype(dtype)
NumOp_matrix = pauli_str_to_matrix(NumOp_Pauli, N).astype(dtype)
Sz_matrix = pauli_str_to_matrix(Sz_Pauli, N).astype(dtype)

# Convert to Tensor type
H_tensor = paddle.to_tensor(H_matrix)
NumOp_tensor = paddle.to_tensor(NumOp_matrix)
Sz_tensor = paddle.to_tensor(Sz_matrix)

# Classical methods to obtain the eigenstates of the Hamiltonian matrix
_, eigenvectors = numpy.linalg.eigh(H_matrix)

start_time = time.time()

# Get desktop path
desktop_path = os.path.join(os.path.expanduser("~"), "Desktop")

# Specify folder path
folder_name = f"Data_2D_{file_corresponding[proc_sel]}"
folder_path = os.path.join(desktop_path, folder_name)

# Create folder (if the folder does not exist)
os.makedirs(folder_path, exist_ok=True)

for file_index in range(1, n_file + 1):
    file_value = os.path.join(folder_path, f"{file_corresponding[proc_sel]}_{file_index}.txt")
    file_params = os.path.join(folder_path, f"{file_corresponding[proc_sel]}_params_{file_index}.npy")

    with open(file_value, 'w') as file:

        # Create an instance
        VQE_symmetry = VQE_symmetry_class(depth=D, 
                                          width=W, 
                                          length_layer=L)

        # Adam optimizer
        optimizer = paddle.optimizer.Adam(learning_rate=Learning_Rate, 
                                          parameters=VQE_symmetry.parameters())

        # Optimization process
        for iter in range(1, iterations + 1):
            loss, H_psi, NumOp_psi, Sz_psi, cir = VQE_symmetry(proc_sel, 
                                                               H_tensor, 
                                                               NumOp_tensor, 
                                                               Sz_tensor)

            loss.backward()
            optimizer.minimize(loss)
            optimizer.clear_grad()

            # Save results to file
            selected_indices = states_corresponding[proc_sel]
            if iter % iter_step_size == 0 and proc_sel not in [Hea_g, Spa_g]:
                file.write(f'iterations: {iter}, loss: {loss.numpy()[0]:.{prec}f}\n')
            
            for i in range(6):
                if iter % iter_step_size == 0 and i in selected_indices:
                    file.write(f'iterations: {iter}, NumOp_psi{i}: {NumOp_psi[i].numpy()[0]:.{prec}f}\n')

            for i in range(6):
                if iter % iter_step_size == 0 and i in selected_indices:
                    file.write(f'iterations: {iter}, Sz_psi{i}: {Sz_psi[i].numpy()[0]:.{prec}f}\n')

            for i in range(6):
                if iter % iter_step_size == 0 and i in selected_indices:
                    file.write(f'iterations: {iter}, E{i}: {H_psi[i].numpy()[0]:.{prec}f}\n')

            # Compute fidelity
            U_theta = cir.unitary_matrix()

            ground_state = numpy.dot(U_theta, initial_state_g)

            initial_states = [globals()[f'initial_state_e{i}'] for i in range(1, 6)]
            excited_states = [numpy.dot(U_theta, initial_state_e) for initial_state_e in initial_states]

            ground_state_dagger = numpy.conj(ground_state)
            excited_states_dagger = [numpy.conj(state) for state in excited_states]

            fidelities = []

            inner_product_g = numpy.vdot(ground_state_dagger, eigenvectors[:, 0])
            fidelity_g = abs(inner_product_g)**2
            fidelities.append(fidelity_g)

            for i in range(5):
                inner_product_e = numpy.vdot(excited_states_dagger[i], eigenvectors[:, i + 1])
                fidelity_e = abs(inner_product_e)**2
                fidelities.append(fidelity_e)

            if iter % iter_step_size == 0 and proc_sel in [Hea_g, Spa_g]:
                file.write(f'iterations: {iter}, Fidelity of ground state: {fidelity_g:.{prec_fid}f}\n')
            elif iter % iter_step_size == 0 and proc_sel in [Hea_ae, Hea_ae_Sz]:
                file.write(f'iterations: {iter}, Fidelity of ground state: {fidelity_g:.{prec_fid}f}\n')
                for i in range(1, 6):
                    if i in selected_indices:
                        file.write(f'iterations: {iter}, Fidelity of excited state {i}: {fidelities[i]:.{prec_fid}f}\n')
            elif iter % iter_step_size == 0:
                for i in range(1, 6):
                    if i in selected_indices:
                        file.write(f'iterations: {iter}, Fidelity of excited state {i}: {fidelities[i]:.{prec_fid}f}\n')

        params_unrounded = VQE_symmetry.parameters()[0].numpy()
        params_prec = numpy.round(params_unrounded, decimals=8)    # parameters precision
        numpy.save(file_params, params_prec)

# runtime
end_time = time.time()
elapsed_time = end_time - start_time
print(f'Code execution time: {elapsed_time:.4f} seconds')


C:\software\Anaconda\envs\paddle_quantum_env\lib\site-packages\paddle\tensor\creation.py:125: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if data.dtype == np.object:
C:\software\Anaconda\envs\paddle_quantum_env\lib\site-packages\paddle\tensor\creation.py:125: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if data.dtype == np.object:


Select the process you want to compute. You can choose from (copy one of the lines):

Hardware efficient ansatz, ground state
Hardware efficient ansatz, all eigenstates
Hardware efficient ansatz, all eigenstates, Sz penalty term
Symmetry preserving ansatz, ground state
Symmetry preserving ansatz, Eigenstate, n=3
Symmetry preserving ansatz, Eigenstate, n=4

Note: If any other parameters need to be modified, please adjust them in the code. Symmetry preserving ansatz, Eigenstate, n=4


You selected: Symmetry preserving ansatz, Eigenstate, n=4

Program is running...     The generated data is saved in the Data_2D_Spa_en4 folder on the desktop.



C:\software\Anaconda\envs\paddle_quantum_env\lib\site-packages\paddle\fluid\framework.py:1104: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  elif dtype == np.bool:


Code execution time: 16555.9293 seconds
